In [1]:
# import all the things
# API Key: 46b582e264cd589807d0c7d4fef9e732
# url: https://api.themoviedb.org/3

import json
import csv
import pandas as pd
import requests
import tmdbsimple as tmdb
import timeit
import numpy as np

tmdb.API_KEY = '46b582e264cd589807d0c7d4fef9e732'
API_KEY = '46b582e264cd589807d0c7d4fef9e732'
tmdb_url = 'https://api.themoviedb.org/3'

In [2]:
# sanity check: Looking for a '200' status code to return

tmdb_request = requests.get('https://api.themoviedb.org/3/movie/76341?api_key={}'.format(API_KEY))
tmdb_request.status_code

200

In [3]:
# obtain the daily file export of collections
# this returns a status.
# copy and paste the link into a browser and a download of the .gz file will begin.
# Open with 7-zip console and extract

requests.get('https://files.tmdb.org/p/exports/collection_ids_01_13_2023.json.gz')

<Response [200]>

In [4]:
# bring in the json file

collection_ids = pd.read_json("C:/Users/tando/OneDrive/General Assembly/Capstone/JSON Files/collection_ids_01_13_2023.json", lines=True)

In [5]:
# sanity check: take a look at collection_ids

collection_ids

,id,name
0,10,Star Wars Collection
1,84,Indiana Jones Collection
2,119,The Lord of the Rings Collection
3,131,Trois Couleurs Collection
4,151,Star Trek: The Original Series Collection
...,...,...
5755,1073118,Sordid Lives Collection
5756,1073188,Anal Models Collection
5757,1073243,First Anal Collection
5758,1073290,Rascal Does Not Dream of Collection


In [6]:
# create an array of id values

collection_ids_only = collection_ids['id'][ : ].values

In [7]:
#sanity check: check the variable type to make sure it is an array

collection_ids_only

array([     10,      84,     119, ..., 1073243, 1073290, 1073293],
      dtype=int64)

In [8]:
# sanity check: check min id of array

ids_min = min(collection_ids_only)
ids_min

10

In [9]:
#sanity check: check max id of array

ids_max = max(collection_ids_only)
ids_max

1073293

In [13]:
# find out length of collection_ids_only list so you know how many API calls to make in the loop below

len(collection_ids_only)

5760

In [14]:
# create an empty list where API calls will populate
# DO NOT RE-RUN THIS CELL AFTER POPULATING THE LIST WITH API CALLS BELOW (unless you want to start over)

teri_collection_list = []

In [50]:
# loop to make API calls
# increments of 500
# returned error for list item 5139, collection id #1007775 does not exist
# returned error for list item 5375, collection id #1040122 does not exist 
# returned error for list item 5376, collection id #1040388 does not exist
# returned error for list item 5566, collection id #1063117 does not exist

for i in collection_ids_only [5568 : 5760]:
    collection = tmdb.Collections(i)
    response = collection.info()
    teri_collection_list.append(response)


In [51]:
# check list length (make sure it matches length of collection_ids_only)

len(teri_collection_list)

5757

In [52]:
#sanity check: make sure it's a list

type(teri_collection_list)

list

In [54]:
# convert list to a dataframe

collection_df = pd.DataFrame(columns=['id', 'name', 'overview', 'poster_path', 'backdrop_path', 'parts'])

for i in range(len(teri_collection_list)):
    normalized_df = pd.json_normalize(teri_collection_list[i])
    collection_df = pd.concat([collection_df, normalized_df], ignore_index = True)

In [63]:
# sanity check: look at the dataframe

collection_df.head(2)

,id,name,overview,poster_path,backdrop_path,parts
0,10,Star Wars Collection,"An epic space-opera theatrical film series, wh...",/gq5Wi7i4SF3lo4HHkJasDV95xI9.jpg,/d8duYyyC9J5T825Hg7grmaabfxQ.jpg,"[{'adult': False, 'backdrop_path': '/aDYSnJAK0..."
1,84,Indiana Jones Collection,Adventure film series created by George Lucas ...,/lpxDrACKJhbbGOlwVMNz5YCj6SI.jpg,/6kh59mZizZsttZPR0HAdXk6Ve2n.jpg,"[{'adult': False, 'backdrop_path': '/c7Mjuip0j..."


In [64]:
# drop unneccesary columns

collection_df.drop(columns = ['overview', 'poster_path', 'backdrop_path'], inplace = True)

In [65]:
# sanity check: the above three columns should be dropped

collection_df.head(2)

,id,name,parts
0,10,Star Wars Collection,"[{'adult': False, 'backdrop_path': '/aDYSnJAK0..."
1,84,Indiana Jones Collection,"[{'adult': False, 'backdrop_path': '/c7Mjuip0j..."


In [66]:
# explode 'parts'

collection_parts = collection_df.explode('parts', ignore_index = True)

In [67]:
# check new length of df

len(collection_parts)

23110

In [68]:
# sanity check: the first collection should be multiple rows now that 'parts' is exploded

collection_parts.head(4)

,id,name,parts
0,10,Star Wars Collection,"{'adult': False, 'backdrop_path': '/aDYSnJAK0B..."
1,10,Star Wars Collection,"{'adult': False, 'backdrop_path': '/dMZxEdrWIz..."
2,10,Star Wars Collection,"{'adult': False, 'backdrop_path': '/soJdiy91Mc..."
3,10,Star Wars Collection,"{'adult': False, 'backdrop_path': '/5fu7fzy4NZ..."


In [72]:
# check for NaN values (eg, collections without movie ids)

collection_parts.isna().sum()

id       0
name     0
parts    0
dtype: int64

In [73]:
#remove NaN rows

collection_parts.dropna(inplace=True)

In [75]:
# sanity check: df length should now be less if NaN were dropped

len(collection_parts)

23082

In [76]:
# sanity check: isolate location of movie id within 'parts'

collection_parts['parts'][0]['id']

11

In [78]:
# export collections df as csv

collection_parts.to_csv('collection_parts_df_011423.csv')

In [79]:
# now, we want to isolate JUST the movie ids into its own dataframe so we can use them to make movie ID API calls
# create a new list to populate

teri_valid_ids = []

In [80]:
# we create a loop to extract movie IDs from 'parts'
for i in range(len(collection_parts['parts'])):
    valid_parts = collection_parts.iloc[i, 2]['id']
    teri_valid_ids.append(valid_parts)

In [81]:
# sanity check: check the length of the new list

len(teri_valid_ids)

23082

In [82]:
# create df of valid IDs

collection_valid_ids_df = pd.DataFrame(teri_valid_ids)

In [83]:
# sanity check: look at the new dataframe. Should just be one column and one index.

collection_valid_ids_df.head(5)

,0
0,11
1,1891
2,1892
3,1893
4,1894


In [84]:
# export df of valid ids as csv

collection_valid_ids_df.to_csv('collection_valid_ids_df_011423.csv')

In [86]:
# sanity check: sometimes these csvs import an extra column when reloading as df. 
# import the df to see if the extra column exists
# if it does, open the csv and remove the first ',' then save and re-import

quick_peek = pd.read_csv('./collection_valid_ids_df_011423.csv')
quick_peek

,0
0,11
1,1891
2,1892
3,1893
4,1894
...,...
23077,795245
23078,1056803
23079,572154
23080,283069


In [87]:
# Now, you're ready to use this list of movie IDs to make your API calls!
# First, make a new list

valid_movie_list = []

In [137]:
# run your for loop through the valid IDs to populate your new movie list
# increments of 1000
# returned error for list item 867, movie id #19262 does not exist
# returned error for list item 1137, movie id #50252 does not exist
# returned error for list item 9868, movie id #15169 does not exist

for i in teri_valid_ids[20000 : 23082]:
    movies = tmdb.Movies(i)
    response = movies.info()
    valid_movie_list.append(response)

In [138]:
# check list length
# final list length:

len(valid_movie_list)

23081

In [139]:
# sanity check: type

type(valid_movie_list)

list

In [140]:
# sanity check: return index 0

valid_movie_list[0]

{'adult': False,
 'backdrop_path': '/aDYSnJAK0BTVeE8osOy22Kz3SXY.jpg',
 'belongs_to_collection': {'id': 10,
  'name': 'Star Wars Collection',
  'poster_path': '/gq5Wi7i4SF3lo4HHkJasDV95xI9.jpg',
  'backdrop_path': '/d8duYyyC9J5T825Hg7grmaabfxQ.jpg'},
 'budget': 11000000,
 'genres': [{'id': 12, 'name': 'Adventure'},
  {'id': 28, 'name': 'Action'},
  {'id': 878, 'name': 'Science Fiction'}],
 'homepage': 'http://www.starwars.com/films/star-wars-episode-iv-a-new-hope',
 'id': 11,
 'imdb_id': 'tt0076759',
 'original_language': 'en',
 'original_title': 'Star Wars',
 'overview': 'Princess Leia is captured and held hostage by the evil Imperial forces in their effort to take over the galactic Empire. Venturesome Luke Skywalker and dashing captain Han Solo team together with the loveable robot duo R2-D2 and C-3PO to rescue the beautiful princess and restore peace and justice in the Empire.',
 'popularity': 85.36,
 'poster_path': '/6FfCtAuVAW8XJjZ7eWeLibRLWTw.jpg',
 'production_companies': [{'id'

In [164]:
# assign new variables to the datafram you're about to create

adult = 'adult'
belongs_to_collection = 'belongs_to_collection'
budget = 'budget'
genres = 'genres'
movie_id = 'id'
original_language = 'original_language'
original_title = 'original_title'
popularity = 'popularity'
release_date = 'release_date'
revenue = 'revenue'
runtime = 'runtime'
status = 'status'
title = 'title'
vote_average = 'vote_average'
vote_count = 'vote_count'

In [165]:
# create dataframe

movie_collection_df = pd.DataFrame(columns=['adult', 'belongs_to_collection', 'budget', 'genres', 'id', 'original_language', 'original_title', 'popularity', 'release_date', 'revenue', 'runtime',
                                           'status', 'title', 'vote_average', 'vote_count'])

In [166]:
# sanity check: pull up the df. should be empty

movie_collection_df

,adult,belongs_to_collection,budget,genres,id,original_language,original_title,popularity,release_date,revenue,runtime,status,title,vote_average,vote_count


In [167]:
# populate the dataframe with data from the movie list
# this is returning an error when it runs into an entry that is missing information. it will take some time. 

for i in range(len(valid_movie_list)):
    d = {adult : valid_movie_list[i]['adult'],
         belongs_to_collection : valid_movie_list[i]['belongs_to_collection'],
         budget : valid_movie_list[i]['budget'],
         genres : valid_movie_list[i]['genres'],
         movie_id : valid_movie_list[i]['id'],
         original_language: valid_movie_list[i]['original_language'],
         original_title : valid_movie_list[i]['original_title'], 
         popularity : valid_movie_list[i]['popularity'],
         release_date : valid_movie_list[i]['release_date'],
         revenue : valid_movie_list[i]['revenue'],
         runtime : valid_movie_list[i]['runtime'],
         status : valid_movie_list[i]['status'],
         title : valid_movie_list[i]['title'],
         vote_average : valid_movie_list[i]['vote_average'],
         vote_count : valid_movie_list[i]['vote_count']}
    
    row = pd.DataFrame([d])
    movie_collection_df = pd.concat([movie_collection_df, row], ignore_index = True)

In [168]:
# sanity check: check the first few rows of the dataframe

movie_collection_df.head(4)

,adult,belongs_to_collection,budget,genres,id,original_language,original_title,popularity,release_date,revenue,runtime,status,title,vote_average,vote_count
0,False,"{'id': 10, 'name': 'Star Wars Collection', 'po...",11000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",11,en,Star Wars,85.360,1977-05-25,775398007,121,Released,Star Wars,8.206,18120
1,False,"{'id': 10, 'name': 'Star Wars Collection', 'po...",18000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",1891,en,The Empire Strikes Back,27.556,1980-05-20,538400000,124,Released,The Empire Strikes Back,8.393,14941
2,False,"{'id': 10, 'name': 'Star Wars Collection', 'po...",32350000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",1892,en,Return of the Jedi,26.433,1983-05-25,572700000,135,Released,Return of the Jedi,7.886,13757
3,False,"{'id': 10, 'name': 'Star Wars Collection', 'po...",115000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",1893,en,Star Wars: Episode I - The Phantom Menace,31.762,1999-05-19,924317558,136,Released,Star Wars: Episode I - The Phantom Menace,6.518,12828


In [169]:
# to avoid returning errors for NaN values, assign a fake name to missing entries in collection field

movie_collection_df['collection_name'] = 'fake_name'

In [170]:
for i in range(len(movie_collection_df.belongs_to_collection)):
    if movie_collection_df.iloc[i, 1] == None:
        movie_collection_df.iloc[i, 1] = 'some_collection'
    else:
        movie_collection_df.iloc[i, -1] = movie_collection_df.iloc[i, 1]['name']
        movie_collection_df.iloc[i, 1] = movie_collection_df.iloc[i, 1]['id']

In [171]:
# explode genres

movie_collection_df = movie_collection_df.explode('genres', ignore_index = True)

In [172]:
# sanity check

movie_collection_df.head(5)

,adult,belongs_to_collection,budget,genres,id,original_language,original_title,popularity,release_date,revenue,runtime,status,title,vote_average,vote_count,collection_name
0,False,10,11000000,"{'id': 12, 'name': 'Adventure'}",11,en,Star Wars,85.360,1977-05-25,775398007,121,Released,Star Wars,8.206,18120,Star Wars Collection
1,False,10,11000000,"{'id': 28, 'name': 'Action'}",11,en,Star Wars,85.360,1977-05-25,775398007,121,Released,Star Wars,8.206,18120,Star Wars Collection
2,False,10,11000000,"{'id': 878, 'name': 'Science Fiction'}",11,en,Star Wars,85.360,1977-05-25,775398007,121,Released,Star Wars,8.206,18120,Star Wars Collection
3,False,10,18000000,"{'id': 12, 'name': 'Adventure'}",1891,en,The Empire Strikes Back,27.556,1980-05-20,538400000,124,Released,The Empire Strikes Back,8.393,14941,Star Wars Collection
4,False,10,18000000,"{'id': 28, 'name': 'Action'}",1891,en,The Empire Strikes Back,27.556,1980-05-20,538400000,124,Released,The Empire Strikes Back,8.393,14941,Star Wars Collection


In [173]:
# assign missing genre values as no_genre

movie_collection_df.genres = movie_collection_df.genres.fillna('no_genre')

In [174]:
# iterate through the genres list to extract name only

for i in range(len(movie_collection_df.genres)):
    if movie_collection_df.iloc[i, 3] == 'no_genre':
        continue
    else:
        movie_collection_df.iloc[i, 3] = movie_collection_df.iloc[i, 3]['name']

In [175]:
# sanity check

movie_collection_df.head(5)

,adult,belongs_to_collection,budget,genres,id,original_language,original_title,popularity,release_date,revenue,runtime,status,title,vote_average,vote_count,collection_name
0,False,10,11000000,Adventure,11,en,Star Wars,85.360,1977-05-25,775398007,121,Released,Star Wars,8.206,18120,Star Wars Collection
1,False,10,11000000,Action,11,en,Star Wars,85.360,1977-05-25,775398007,121,Released,Star Wars,8.206,18120,Star Wars Collection
2,False,10,11000000,Science Fiction,11,en,Star Wars,85.360,1977-05-25,775398007,121,Released,Star Wars,8.206,18120,Star Wars Collection
3,False,10,18000000,Adventure,1891,en,The Empire Strikes Back,27.556,1980-05-20,538400000,124,Released,The Empire Strikes Back,8.393,14941,Star Wars Collection
4,False,10,18000000,Action,1891,en,The Empire Strikes Back,27.556,1980-05-20,538400000,124,Released,The Empire Strikes Back,8.393,14941,Star Wars Collection


In [176]:
# export df as csv
movie_collection_df.to_csv('movie_collection_df_011423_final.csv')

In [179]:
# if extra unnamed column is included, remove "," in csv file directly
# or, https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html

quick_look = pd.read_csv('./movie_collection_df_011423_final.csv')

In [189]:
quick_look.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45211 entries, 0 to 45210
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45211 non-null  bool   
 1   belongs_to_collection  45211 non-null  object 
 2   budget                 45211 non-null  int64  
 3   genres                 45211 non-null  object 
 4   id                     45211 non-null  int64  
 5   original_language      45211 non-null  object 
 6   original_title         45211 non-null  object 
 7   popularity             45211 non-null  float64
 8   release_date           44613 non-null  object 
 9   revenue                45211 non-null  int64  
 10  runtime                45211 non-null  int64  
 11  status                 45211 non-null  object 
 12  title                  45211 non-null  object 
 13  vote_average           45211 non-null  float64
 14  vote_count             45211 non-null  int64  
 15  co

In [181]:
# CONGRATS!!!!
# data is ready for cleaning and analysis!

In [184]:
# extra step: save df as pickle

movie_collection_df.to_pickle('movie_data_011423.pkl')

In [185]:
test_pickle = pd.read_pickle('movie_data_011423.pkl')

In [186]:
test_pickle

,adult,belongs_to_collection,budget,genres,id,original_language,original_title,popularity,release_date,revenue,runtime,status,title,vote_average,vote_count,collection_name
0,False,10,11000000,Adventure,11,en,Star Wars,85.360,1977-05-25,775398007,121,Released,Star Wars,8.206,18120,Star Wars Collection
1,False,10,11000000,Action,11,en,Star Wars,85.360,1977-05-25,775398007,121,Released,Star Wars,8.206,18120,Star Wars Collection
2,False,10,11000000,Science Fiction,11,en,Star Wars,85.360,1977-05-25,775398007,121,Released,Star Wars,8.206,18120,Star Wars Collection
3,False,10,18000000,Adventure,1891,en,The Empire Strikes Back,27.556,1980-05-20,538400000,124,Released,The Empire Strikes Back,8.393,14941,Star Wars Collection
4,False,10,18000000,Action,1891,en,The Empire Strikes Back,27.556,1980-05-20,538400000,124,Released,The Empire Strikes Back,8.393,14941,Star Wars Collection
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,False,1073293,0,Fantasy,283069,ja,DAICONⅢ オープニングアニメ,2.422,1981-08-22,0,5,Released,DAICON III Opening Animation,7.100,9,Daicon Opening Animation Collection
45207,False,1073293,0,Animation,99662,ja,DAICONⅣ オープニングアニメ,2.948,1983-08-20,0,6,Released,DAICON IV Opening Animation,7.300,20,Daicon Opening Animation Collection
45208,False,1073293,0,Action,99662,ja,DAICONⅣ オープニングアニメ,2.948,1983-08-20,0,6,Released,DAICON IV Opening Animation,7.300,20,Daicon Opening Animation Collection
45209,False,1073293,0,Science Fiction,99662,ja,DAICONⅣ オープニングアニメ,2.948,1983-08-20,0,6,Released,DAICON IV Opening Animation,7.300,20,Daicon Opening Animation Collection


In [187]:
test_pickle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45211 non-null  object 
 1   belongs_to_collection  45211 non-null  object 
 2   budget                 45211 non-null  object 
 3   genres                 45211 non-null  object 
 4   id                     45211 non-null  object 
 5   original_language      45211 non-null  object 
 6   original_title         45211 non-null  object 
 7   popularity             45211 non-null  float64
 8   release_date           45211 non-null  object 
 9   revenue                45211 non-null  object 
 10  runtime                45211 non-null  object 
 11  status                 45211 non-null  object 
 12  title                  45211 non-null  object 
 13  vote_average           45211 non-null  float64
 14  vote_count             45211 non-null  object 
 15  co